# Gap Framework - Computer Vision

In this session, we will introduce you to preprocessing image data for computer vision. Preprocessing, storage, retrieval and batch management are all handled by two classes, the Image and Images class.

    Image - represents a single preprocessed image
    Images - represents a collection (or batch) of preprocessed images

In [1]:
import os
os.chdir("../")

Let's start by importing the Gap Vision module

In [2]:
# import the Gap Vision module
from image import Image, Images

In [3]:
!cd

C:\Users\'\Desktop\epipog-nlp


Let's go to a respository of images for sign language. We will use this repository for image preprocessing for computer vision.

In [4]:
os.chdir("../Training/AITraining/Intermediate/Machine Learning/sign-lang")

In [5]:
# The sign language characters (a-z) are labeled 1 .. 26, and 0 is for not a character.
# Each of the training images are under a subdirectory of the corresponding label.
labels = os.listdir("gestures")
    

In [7]:
# Prepare each set of labeled data into machine learning ready data
# The images are 50x50, bitdepth=8, 1 channel (grayscale)
total = 0
collections=[]
for label in labels:
    # Get a list of all images in the subdirectory for this label (should be 1200 images)
    imgdir = "gestures/" + label + "/"
    imglst = [imgdir + x for x in os.listdir(imgdir)]
    images = Images(imglst, int(label), dir='tmp' + label, config=['flatten', 'grayscale'])
    collections.append(images)
    print("Procesed: " + label, "Number of images:", len(images), "Time: ", images.time)
    total += images.time
    
print("average:", total / len(labels))
    

Procesed: 0 Number of images: 1200 Time:  2.1700029373168945
Procesed: 1 Number of images: 1200 Time:  2.1200029850006104
Procesed: 10 Number of images: 1200 Time:  2.342005729675293
Procesed: 11 Number of images: 1200 Time:  2.130002975463867
Procesed: 12 Number of images: 1200 Time:  2.130002975463867
Procesed: 13 Number of images: 1200 Time:  1.9700028896331787
Procesed: 14 Number of images: 1200 Time:  1.9920039176940918
Procesed: 15 Number of images: 1200 Time:  2.020002841949463
Procesed: 16 Number of images: 1200 Time:  1.9700026512145996
Procesed: 17 Number of images: 1200 Time:  1.9800026416778564
Procesed: 18 Number of images: 1200 Time:  2.0120041370391846
Procesed: 19 Number of images: 1200 Time:  2.650003671646118
Procesed: 2 Number of images: 1200 Time:  5.540007829666138
Procesed: 20 Number of images: 1200 Time:  4.58000636100769
Procesed: 21 Number of images: 1200 Time:  4.480006456375122
Procesed: 22 Number of images: 1200 Time:  5.210007190704346
Procesed: 23 Number o

Let's verify the preprocessing of our image data

In [8]:
# Let's see how many batches (collections) we have (hint: should be 27)
print(len(collections))

# Let's verify that the items in the collections are an Images object
collection = collections[3]
print(type(collection))

# For a collection, let's see how many image objects we have (hint: should be 1200)
print(len(collection))

27
<class 'image.Images'>
1200


Let's look at the first Image object in this collection.

In [9]:
# Let's get the first Image item and verify it is an Image object
image = collection[0]
print(type(image))

<class 'image.Image'>


In [10]:
# Let's get some basic information about the image
print(image.name)  # the root name of the image
print(image.type)  # the image file suffix
print(image.size)  # the size of the image on disk

1
jpg
1459


In [11]:
# Let's now check the raw (uncompressed) unprocessed image
print(image.raw.shape)

(50, 50)


In [12]:
# Let's look at how the image got processed.
print(image.shape)  # Note, that the preprocessed image was flattened into a 1D vector. It was 50x50, and now is 2500.

(2500,)


In [13]:
# Let's view the raw image
import cv2
cv2.imshow('image',image.raw)
cv2.waitKey(0)

-1

## Let's presume that it is some later date, and we want to recall the collections from disk.

We start by creating an empty Images() object. We then identify the name of the collection and execute the load() method with the collection name.

In [14]:
# Let's load up one collection
images = Images(dir="tmp0/")
images.load("batch.1")

Wow, we just recalled the whole collection from disk. Let's now look at our machine learning ready data!

In [15]:
print(len(images))
image = images[17]
print(image.name)
print(image.type)
print(image.size)
print(image.shape)

1200
1013
jpg
369
(2500,)


In [16]:
print(image.raw.shape)

(50, 50)


# End of Session 3